-use the variables and zip text files to modify output


In [1]:
import pandas as pd
import requests
import json

In [2]:
#create list of zipcodes from txt file with zips
zips = open('chosen_zips.txt', 'r').readlines()
zips = [z.replace('\n', '') for z in zips]
zips = ','.join(zips)

In [3]:
#census api key
apiKey = ""

In [4]:
print(zips)

07059,07060,07062,07063,07069,07920,07922,07933,07946,07980,08807,08812,08836,07701,07702,07703,07704,07716,07718,07721,07730,07732,07733,07734,07735,07737,07738,07739,07748,07757,07748,07760,08873,08854,08844,08817,08820,08876,07076


In [5]:
#same thing with variables
variables = open('variables_2018.txt', 'r').readlines()
variables = [z.replace('\n', '') for z in variables]
variables = ','.join(variables)


In [6]:
#just to check for errors, if a ',' appears at the end you need to delete the last blank line in the txt file
print(variables)
#the first variable here corresponds to the last value in columnNames

DP02_0066PE,DP04_0134E,DP04_0102E,DP04_0101E,DP04_0090PE,DP04_0046PE,DP02_0014PE,DP02_0014E,DP02_0013PE,DP02_0013E,DP02_0020PE,DP02_0020E,DP02_0010PE,DP02_0010E,DP02_0003PE,DP02_0003E,DP02_0001E,DP02_0016E,DP02_0015E,DP04_0089E,DP03_0062E,DP03_0119PE,DP03_0009PE,DP02_0065PE,DP02_0065E,DP02_0064PE,DP02_0064E,DP02_0061PE,DP02_0061E,DP05_0018E,DP05_0024E,DP02_0055E,DP05_0021E,DP05_0005E,DP05_0001E


In [7]:
#name your variables, 'zipcode' stays unchanged
#again, note that columnNames is flipped
columnNames = ['zipcode', 'pop', 'under5', '18plus', 'elemschool', '65plus', 'med_age', 'hschool', 'p_hschool', 'bachelors', 'p_bachelors', 'advanced', 'p_advanced', 'unemp',
               'pov', 'med_hholdinc', 'med_homeval', 'avg_hhold', 'avg_fam', 'hhold', 'famhhold', 'p_famhhold', '!famhhold', 'p_!famhhold', 'childpop', 'p_childpop', 'hholdunder18',
               'p_hholdunder18', 'hhold65plus', 'p_hhold65plus', 'p_owneroccup', 'p_oomortgages', 'med_smocmortgages', 'med_smoc!mortgages', 'med_rent', 'p_hschoolplus']
               

In [8]:
#base api, this is where you change the year and dataset
#it could be cleaned a little more with %s
#note it is currently the 2018/acs/acs5/profile dataset
baseAPI = "https://api.census.gov/data/2018/acs/acs5/profile?key=%s&get=%s&for=zip%%20code%%20tabulation%%20area:%s"

In [9]:
#custom API constructed
calledAPI = baseAPI % (apiKey, variables, zips)

In [10]:
#call API and collect the response
response = requests.get(calledAPI)

In [2]:
#check if all the variables are valid, if ok it will just print, if not there will be an error
#print(response.text)

In [3]:
#load the response into a JSON, ignoring the first element which is just field labels
formattedResponse = json.loads(response.text)[1:]
#print(formattedResponse)

NameError: name 'json' is not defined

In [13]:
#flip the order of the response from [population, zipcode] -> [zipcode, population]
#this is kind of annoying since it affects the entire column name order
formattedResponse = [item[::-1] for item in formattedResponse]

In [14]:
#store the response in a dataframe
#remember the last column name is assigned to the first line in the variable text file
df = pd.DataFrame(columns=columnNames, data=formattedResponse)

In [15]:
df.head()

,zipcode,pop,under5,18plus,elemschool,65plus,med_age,hschool,p_hschool,bachelors,...,hholdunder18,p_hholdunder18,hhold65plus,p_hhold65plus,p_owneroccup,p_oomortgages,med_smocmortgages,med_smoc!mortgages,med_rent,p_hschoolplus
0,07980,2253,200,1701,231,359,41.5,203,13.1,359,...,259,31.8,246,30.2,78.8,642,3170,130,1381,95.7
1,07704,5943,372,3912,1026,683,40.8,332,9.1,1559,...,990,53.1,461,24.7,90.2,1682,3539,357,2625,98.2
2,07701,23861,1358,18888,2023,4315,41.7,3590,20.8,4691,...,2702,27.6,3277,33.4,66.7,6536,2638,2186,1517,88.9
3,07757,5667,408,4502,456,1012,48.6,941,22.8,1440,...,560,26.7,776,36.9,85.9,1804,2534,497,1250,96.3
4,07718,7139,358,5431,805,850,37.7,1715,36.0,1135,...,934,39.8,598,25.5,81.2,1906,2445,622,1660,93.4


In [1]:
#check dtypes
#df.dtypes
#convert as neccisary

In [17]:
#converting them to_numeric
#here I am converting everything after zipcode, the first column
colstoconvert = columnNames[1:]
df[colstoconvert] = df[colstoconvert].apply(pd.to_numeric, errors='coerce')


In [19]:
#not used but here for reference
#converting zipcode back to string
#somersetZipPopulations['zipcode'] = somersetZipPopulations['zipcode'].astype(str)
#somersetZipPopulations.dtypes

Now let's add the columns for the percentage of people 65 and older (p65+) and for school age children 5-17 (pSchoolAge)

In [20]:
#first the p65+
df['p_65plus'] = (df['65plus']/df['pop'])

In [21]:
#now SchoolAge and then pSchoolAge
df['schoolage'] = df['pop'] - df['18plus']
df['schoolage'] = df['schoolage'] - df['under5']
df['p_schoolage'] = (df['schoolage'] / df['pop'])

#percent without a high school diploma
df['p_!hschool'] = df['p_hschoolplus'].apply(lambda x: 100-x)

#add a year column
df['year'] = '2018'

In [22]:
#convert whole number percentage columns to decimal so excel can apply percent formatting
pcolumns = ['p_hschool', 'p_bachelors', 'p_advanced', 'unemp', 'pov', 'p_famhhold', 'p_!famhhold', 'p_childpop',
            'p_hholdunder18', 'p_hhold65plus', 'p_owneroccup', 'p_oomortgages', 'p_hschoolplus', 'p_!hschool']

temp = df[pcolumns].apply(lambda x: x/100)

df = df.drop(pcolumns, axis=1)

df = df.join(temp)

In [23]:
df.head()

,zipcode,pop,under5,18plus,elemschool,65plus,med_age,hschool,bachelors,advanced,...,pov,p_famhhold,p_!famhhold,p_childpop,p_hholdunder18,p_hhold65plus,p_owneroccup,p_oomortgages,p_hschoolplus,p_!hschool
0,07980,2253,200,1701,231,359,41.5,203,359,457,...,0.021,0.318,0.229,0.369,0.318,0.302,0.788,6.42,0.957,0.043
1,07704,5943,372,3912,1026,683,40.8,332,1559,1069,...,0.036,0.522,0.113,0.410,0.531,0.247,0.902,16.82,0.982,0.018
2,07701,23861,1358,18888,2023,4315,41.7,3590,4691,2971,...,0.060,0.253,0.417,0.291,0.276,0.334,0.667,65.36,0.889,0.111
3,07757,5667,408,4502,456,1012,48.6,941,1440,719,...,0.053,0.257,0.198,0.329,0.267,0.369,0.859,18.04,0.963,0.037
4,07718,7139,358,5431,805,850,37.7,1715,1135,528,...,0.028,0.361,0.216,0.368,0.398,0.255,0.812,19.06,0.934,0.066


In [24]:
#assign the names of towns and such
name_df = pd.read_excel("town_names.xlsx", dtype={'zipcode': object})

df = df.set_index('zipcode').join(name_df.set_index('zipcode'))

In [25]:
#save that dataframe to excel
df.to_excel('./acs5_data/acs5_2018_zip.xlsx', index=True)